<a href="https://colab.research.google.com/github/hsgw/keyboard-made-by-python/blob/main/notebook/jp/pcb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 基板の設計
このノートブックではキーボードの基板を設計し、最終的に基板を生産するためのデータ`ガーバーファイル`を得ることを目標とします。

通常の工程で基板を作るにはまず`回路図エディタ`で`回路図`を書いて使用する部品とその結線情報をまとめて`ネットリスト`として出力し、`基板エディタ`を用いて実際に`基板`へ部品を配置・配線します。

## 記事を読むための注意
このノートブックには2回実行すると正常に動作しないセルが含まれています。

それを防止するために上のメニューから`ランタイム>全てのセルを実行`してから読み始めると良いかもしれません。実行完了には少し時間がかかります。

# skidlでネットリスト(回路図)を設計する
Pythonを使った設計では回路図を書かずにコードで使用する部品と結線情報を定義してネットリストを出力します。

[skidl](https://github.com/devbisme/skidl)というライブラリを用います。このライブラリは独自フォーマットの部品ライブラリだけでなく、kicadの部品ライブラリをインポートして使用します。また、ネットリストはkicadのPCBエディタと互換性があり直接kicadへ読み込んで回路図を書くこともできます。

- skidl  https://github.com/devbisme/skidl
- skidlのドキュメント  https://devbisme.github.io/skidl/

## kicadライブラリのダウンロード
以下のセルを実行してkicadのライブラリが入ったリポジトリをクローンしておきます。

In [ ]:
!git clone https://github.com/hsgw/keyboard-made-by-python/

## skidlのインストールとインポート
[skidl](https://github.com/devbisme/skidl)をインストールしてインポートします。

In [ ]:
# インストールには時間がかかります
!pip install skidl

In [ ]:
# kicadのデフォルトパスが環境変数に設定されていないのでWarningが出ます
from skidl import *

## 定数を宣言する
何度も出てくる値を定数として宣言しておきます。この値を使い回すことで変更に強い設計になります。


In [ ]:
KEY_COUNT = 10
COL_COUNT = 4
ROW_COUNT = 3
MATRIX_MAP = [
        (0,1),(0,2),(0,3),  
        (1,1),(1,2),(1,3),
  (2,0),(2,1),(2,2),(2,3)
]

## kicadライブラリの読み込みと部品の準備
使用するkicadライブラリのパスを追加をします。

kicadがインストールされている環境であれば通常のライブラリへのパスがデフォルトで追加されているはずですが、今回はkicadのインストールされていない環境で実行するために使う部品だけ別に用意しておきます。
先程githubからcloneしてきた`keyboard-made-by-python/hardware/kicad_libs`に入っています。

シンボルとフットプリントを結びつけて必要な個数だけ部品を用意します。

In [ ]:
# 使用するライブラリを登録する
# シンボル・フットプリントどちらも別で登録する
lib_search_paths[KICAD].append("keyboard-made-by-python/hardware/kicad_libs")
footprint_search_paths[KICAD].append("keyboard-made-by-python/hardware/kicad_libs/kicad.pretty")

# 同じ部品を何回も使うならシンボルとフットプリントを結びつけたテンプレートとして読み込んでおく
diode = Part(
  "kicad_symbols", "D_Small_ALT", TEMPLATE, footprint="kicad:D_SOD123_hand"
)
switch = Part(
  "kicad_symbols",
  "SW_Push",
  TEMPLATE,
  footprint="kicad:SW_Cherry_MX_1.00u_PCB",
)

# 宣言した定数とテンプレートを使ってダイオードとスイッチを用意する
# それぞれ KEY_COUNT 個用意して配列にいれておく
diodes = diode(KEY_COUNT)
switches = switch(KEY_COUNT)

# ひとつしか使わない部品はそのまま読み込む
xiao = Part("kicad_symbols", "xiao_rp2040", footprint="kicad:xiao_rp2040")
oled = Part("kicad_symbols", "oled_i2c", footprint="kicad:oled_i2c")

# printするとシンボルのPin情報が見えます
print(diode, switch, xiao, oled)

## PinやNetを接続する
用意した部品のピン同士を接続を入力して回路の構成を定義します。

このときなるべく配線自体に`Net`名をつけるようにします。`Net`を基準に部品の`Pin`を繋いでいくことで同じところにつながるもの(電源やスイッチマトリクス)を整理して見やすいコードにします。また、kicadへネットリストをインポートしたときに表示されるので目印にもなります。

今回の設計でも基板配線時に`ピン番号`を`Net`から読み出します。

In [ ]:
# Pin情報を表示しながら配線すると便利
print(xiao)

In [ ]:
# スイッチマトリクスのROW,COLのネットリストが入った配列を作る
netRows = [Net(f"ROW{i}") for i in range(ROW_COUNT)]
netCols = [Net(f"COL{i}") for i in range(COL_COUNT)]


# ROWのNet -> スイッチの1ピン  スイッチの2ピン -> ダイオードのカソード  ダイオードのアノード -> COLのNet をまとめて宣言する
# NetやPinは`&`で繋ぐと接続される
# 部品のPinには part["pin name"]でアクセス出来る
# 部品のPinの添字を2つにするとinとoutになる
# 例: sw["1"]につながるNetかPin & sw["1 2"] & sw["2"]に繋がるNetかPin
for sw, d, mapping in zip(switches, diodes, MATRIX_MAP):
  netRows[mapping[0]] & sw["1 2"] & d["K A"] & netCols[mapping[1]]

# スイッチマトリクスとxiaoを接続する
# NetにPinを`+`ことで接続する
netCols[0] += xiao[8]
netCols[1] += xiao[3]
netCols[2] += xiao[4]
netCols[3] += xiao[5]

netRows[0] += xiao[1]
netRows[1] += xiao[6]
netRows[2] += xiao[7]

# oledとxiaoも接続する
# Net("3.3V")みたいに直接Netを宣言して繋いでもOK
Net("3.3V") & oled["Vcc"] & xiao["3.3V"]
Net("GND") & oled["GND"] & xiao["GND"]
Net("SDA") & oled["SDA"] & xiao[9]
Net("SCL") & oled["SCL"] & xiao[11]

# printすると接続されているPinが表示される
print(netRows[0])

## ERCとネットリストの出力
ERC(Electrical Rule Check・回路図のルールチェック)をかけて、ネットリストを出力します。
これで回路図・ネットリストは完成です！

In [ ]:
# 他のセルを複数回実行しているとエラーが出たり回路が複数個になったりするかもしれません
# そのときは`メニュー`の`ランタイム -> ランタイムを再起動`して最初からやりなおすか`再起動してすべてのセルを実行`してください
# 未結線のWarningが出ますが問題ありません
ERC()
generate_netlist(file_="keyboard.net")

出力されたネットリストは今記事では使用しませんが、kicadのpcbnewのネットリスト読み込みからインポートしてそのまま基板を作ることも出来ます。

![pcbnewへネットリストをインポートした](https://github.com/hsgw/keyboard-made-by-python/blob/main/notebook/imgs/kicad_pcbnew.png?raw=1)

# pcbflowで基板を設計する
[pcbflow](https://github.com/michaelgale/pcbflow)を使って、skidlで設計したフットプリントとピンの接続情報を読み込み、基板上に実際に部品を配置し配線を繋いでいきます。随時、画像ファイルでプレビューをしたり、最終的には製造に必要なガーバーファイル一式を出力します。

配線は昔ながらの[Turtle graphics](https://docs.python.org/ja/3/library/turtle.html)風の表記で行います。

pcbflowは開発中(?)のようで意図しない動作や不具合があったため修正と追加をしました。今記事ではその[folk](https://github.com/hsgw/pcbflow/tree/fix_kicad)を使用します。

- pcbflow https://github.com/michaelgale/pcbflow
- pcvflowのドキュメントはREADME.mdにあります
- folkして修正したpcbflow https://github.com/hsgw/pcbflow/tree/fix_kicad

## pcbflowのダウンロードとインストール
以下のセルを実行してインストールします。
インポートしてエラーがないことを確認します。

In [ ]:
# インストールに時間がかかります
!pip install git+https://github.com/hsgw/pcbflow/@fix_kicad

In [ ]:
from pcbflow import *

## 定数を宣言する
ネットリストと同じようによく使う値を定数として宣言しておきます。   
座標のY軸が反転していて違和感があったので変換するための関数も宣言しています。

In [ ]:
BOARD_WIDTH = 76.0
BOARD_HEIGHT = 57.0

KEY_PITCH = 19.0

SCREW_HOLE = 2.2

LAYER_TOP = "GTL"
LAYER_BOTTOM = "GBL"

# Y軸の座標を反転させる関数
def pos(x, y):
  return (x, BOARD_HEIGHT - y)

## 基板と回路の変数を宣言する
基板と回路にアクセスするための変数を宣言しておきます。
基板はpcbflowのもの、回路はskidlのものです。

`board`の宣言時に外形の大きさを指定し、外形線を追加します。

In [ ]:
# skidlで作った回路情報
circuit = builtins.default_circuit
# pcbflowの基板
board = Board((BOARD_WIDTH, BOARD_HEIGHT))
board.add_outline()

## デザインルールを設定する
ドリルの大きさや配線の太さなど、基板のデザインルールを設定します。

In [ ]:
board.drc.trace_width = 0.5        # 配線の太さ
board.drc.via_drill = 0.6          # ビアのドリル径
board.drc.via_annular_ring = 0.4   # ビアのパッド径
board.drc.clearance = 0.4          # 配線のクリアランス

## 固定用の穴をあける
スイッチの間に基板を固定するための穴を作ります。

In [ ]:
board.add_hole(pos(KEY_PITCH * 2, KEY_PITCH), SCREW_HOLE)
board.add_hole(pos(KEY_PITCH * 3, KEY_PITCH), SCREW_HOLE)
board.add_hole(pos(KEY_PITCH * 3, KEY_PITCH * 2), SCREW_HOLE)
board.add_hole(pos(KEY_PITCH, KEY_PITCH * 2), SCREW_HOLE)

### 画像ファイルを出力してプレビューしてみる
ここで一度画像ファイルを出力して基板上に正しく穴があいているか確認してみます。

In [ ]:
from IPython.display import Image

# 現在のboardをpngファイルとして書き出す
board.save_png("kbd_python", subdir="pcb_png")

# 表示
Image("pcb_png/kbd_python_preview_all.png")
# Image("pcb_png/kbd_python_preview_top.png")
# Image("pcb_png/kbd_python_preview_bot.png")

## 部品に位置情報を追加する
skidlの部品それぞれに基板上の位置・回転・実装面の情報を追加していきます。

今記事ではいきなり位置を指定していますが、基板に配置してはプレビューを繰り返しながら決めていきました。

In [ ]:
# sw, d, xiao, oledはskidlのところで宣言した変数です

# スイッチとダイオードはブロックにまとめて位置を指定する
# ブロックごとにスイッチとダイオードの位置関係は同じ
for sw, d, mapping in zip(switches, diodes, MATRIX_MAP):
  sw.pos = pos(
    mapping[1] * KEY_PITCH + KEY_PITCH / 2,
    mapping[0] * KEY_PITCH + KEY_PITCH / 2,
  )
  sw.side = "top"
  sw.rotate = 0
  d.pos = (sw.pos[0] + 6, sw.pos[1] + 5)
  d.side = "bottom"
  d.rotate = 0

xiao.pos = pos(11.25, 11.5)
xiao.side = "bottom"
xiao.rotate = 0

oled.pos = pos(9.5, 36)
oled.side = "top"
oled.rotate = 270

## 基板に部品を配置する
追加した位置情報を元にskidlの部品をpcbflowの基板へ変換しながら配置していきます。

In [ ]:
for part in circuit.parts:
  try:
    # skidlの部品を読み込んでpcbflowのboardに配置していく
    SkiPart(board.DC(part.pos).right(part.rotate), part, side=part.side)
  except AttributeError:
    print(f"{part.ref} has no pos or side")
    continue

### プレビュー

In [ ]:
from IPython.display import Image

# 現在のboardをpngファイルとして書き出す
board.save_png("kbd_python", subdir="pcb_png")

# 表示
Image("pcb_png/kbd_python_preview_all.png")
# Image("pcb_png/kbd_python_preview_top.png")
# Image("pcb_png/kbd_python_preview_bot.png")

## 配線：xiao-スイッチ(ROW)
xiaoからスイッチのROWへの配線をします。   
Turtle記法を使ってxiaoから配線を引き出してスイッチの`Pad`に接続します。

skidlの`Pin`とpcbflowの`pads`は同じもの、部品のピンを指しています。

In [ ]:
xiaoRef = "U1"

# skidlのNetから名前で検索してPadの番号を返す関数
# skidlのPin番号は1から、pcbflowのPad番号は0から始まる
def get_pin_number_from_net(netLabel, ref):
    net = Net.get(netLabel)
    return list((int(x.num) - 1 for x in net.pins if x.ref == ref))[0]

# skidlのNetから名前で検索して接続されているxiaoのPadを返す関数
# kicadライブラリの都合でPad番号がずれているのを修正して返す
# 最後のnewpath()は初期値に変な値が入っているのを初期化している
def xiao_pads(net: str):
  return (
    board.get_part(xiaoRef)
    .pads[get_pin_number_from_net(net, xiaoRef) + 14]
    .newpath()
  )

# Net:ROWに繋がったxiaoとスイッチのピンを繋ぐ
# ROWに繋がったxiaoのPad→レイヤーを指定→Turtle記法で配線→SWのパッドまで軸を合わせて配線
xiao_pads("ROW0").set_layer(LAYER_TOP).w("r 45").align_meet(
  board.get_part("SW1").pads[0], "x"
)
xiao_pads("ROW1").set_layer(LAYER_TOP).w("r 45").align_meet(
  board.get_part("SW4").pads[0], "y"
)
xiao_pads("ROW2").set_layer(LAYER_TOP).w("l 180 f 12 r 45").align_meet(
  board.get_part("SW8").pads[0], "y"
)

### プレビュー

In [ ]:
from IPython.display import Image

# 現在のboardをpngファイルとして書き出す
board.save_png("kbd_python", subdir="pcb_png")

# 表示
Image("pcb_png/kbd_python_preview_all.png")
# Image("pcb_png/kbd_python_preview_top.png")
# Image("pcb_png/kbd_python_preview_bot.png")

## 配線：ダイオード-スイッチ・スイッチ-スイッチ(ROW)・ダイオード-ダイオード(COL)
接続されているピンの位置関係が同じものであれば同じように接続できるので、ループを回して配線します。

COLの配線では途中でビアを作ります。ビアの情報を保存しておいて後ほどxiaoと接続します。

In [ ]:
# スイッチとダイオードはブロックとして同じように並べたので同じように配線できる
# REF Noでアクセスしたいので1始まり
for i in range(1, KEY_COUNT + 1):
  sw = board.get_part(f"SW{i}")
  d = board.get_part(f"D{i}")
  # 配線をはじめるpadを指定→レイヤーを指定→Turtle記法で配線→パッドと軸を合わせて接続
  d.pads[0].set_layer(LAYER_BOTTOM).w("f 1 r 45").align_meet(sw.pads[1], "x")

# スイッチマトリクスのROWに繋がっているスイッチのPinは直線で並んでいるので同じように配線できる
# ROWによってスイッチの個数が違うので注意
for i in range(3):
  # ROWのNet情報からSWに繋がっているもののRef Noを配列にする
  refs = list(x.ref for x in netRows[i].pins if x.ref.startswith("SW"))
  for j in range(len(refs) - 1):
    # 配線を始めるpadを指定→新しいパスを作る→レイヤーの指定→Turtle記法で配線→指定したpadへ直線で接続
    board.get_part(refs[j]).pads[0].newpath().set_layer(LAYER_TOP).w(
      "r 90 f 2 l 45 f 1 r 45 f 2.5 r 45 f1"
    ).meet(board.get_part(refs[j + 1]).pads[0])

# ビアの情報を保存しておく配列
# xiaoと接続するときに使う
viaCol = [0] * 4

# スイッチマトリクスのCOL1-3に繋がるダイオードは直線に並んでいるので同じように配線できる
# 途中でビアを挟む
for i in range(1, 4):
  d1 = board.get_part(f"D{i}")
  d2 = board.get_part(f"D{i+3}")
  d3 = board.get_part(f"D{i+7}")
  # ビアの情報を保持しておくためにダイオード-ビアを先に配線する
  via = (
    d1.pads[1]
    .set_layer(LAYER_BOTTOM)
    .w("l 180 f 1.25 r 45 f 2 l 45 f 6.5 l 45")
    .align(d2.pads[1], "y")
    .wire()
    .via()
  )
  viaCol[i] = via
  # ビアから次のダイオードのPadまで配線する
  via.set_layer(LAYER_BOTTOM).meet(d2.pads[1])
  d2.pads[1].set_layer(LAYER_BOTTOM).w(
    "l 180 f 1 r 45 f 2 l 45 f 6.5 l 45"
  ).align_meet(d3.pads[1], "y")

### プレビュー

In [ ]:
from IPython.display import Image

# 現在のboardをpngファイルとして書き出す
board.save_png("kbd_python", subdir="pcb_png")

# 表示
Image("pcb_png/kbd_python_preview_all.png")
# Image("pcb_png/kbd_python_preview_top.png")
# Image("pcb_png/kbd_python_preview_bot.png")

## 配線：ビア-xiao(COL)
保存しておいたビアからxiaoへ配線します。

In [ ]:
# ビアからxiaoのpadへ配線する
viaCol[1].set_layer(LAYER_TOP).w("l 45 f 11 l 45").align_meet(
  xiao_pads("COL1"), "x"
)
viaCol[2].set_layer(LAYER_TOP).w("f 2 l 45 f 29.5 l 45").align_meet(
  xiao_pads("COL2"), "x"
)
viaCol[3].set_layer(LAYER_TOP).w("f 4 l 45 f 48.5 l 45").align_meet(
  xiao_pads("COL3"), "x"
)

### プレビュー

## 配線：xiao-OLED
xiaoからOLEDへ配線します。

Netの名前を列挙しておいてskidlのNet情報から接続するPadの番号を調べます。

In [ ]:
oledRef = "DISP1"

# ビアからxiaoのpadへ配線する
for net in ["GND", "3.3V", "SCL", "SDA"]:
  oledPin = board.get_part(oledRef).pads[get_pin_number_from_net(net, oledRef)]
  xiao_pads(net).set_layer(LAYER_TOP).left(135).align_meet(oledPin, "y")

In [ ]:
from IPython.display import Image

# 現在のboardをpngファイルとして書き出す
board.save_png("kbd_python", subdir="pcb_png")

# 表示
Image("pcb_png/kbd_python_preview_all.png")
# Image("pcb_png/kbd_python_preview_top.png")
# Image("pcb_png/kbd_python_preview_bot.png")

### プレビュー

In [ ]:
from IPython.display import Image

# 現在のboardをpngファイルとして書き出す
board.save_png("kbd_python", subdir="pcb_png")

# 表示
Image("pcb_png/kbd_python_preview_all.png")
# Image("pcb_png/kbd_python_preview_top.png")
# Image("pcb_png/kbd_python_preview_bot.png")